In [1]:
import json
import pandas as pd

In [2]:
map_path = "/private/home/arjunrs1/CliMer/scripts/gSAM_test_map.json"
keystep_test_path = "/private/home/arjunrs1/CliMer/data/egoexo4d/egoexo4d_all_views_keysteps_test.csv"

with open(map_path, "rb") as f:
    gsam_map = json.load(f)

keystep_test = pd.read_csv(keystep_test_path)

In [3]:
gsam_map

{'iiith_cooking_65_4_cam02_10': 1811083.0,
 'iiith_cooking_65_4_cam02_8': 234432.0,
 'iiith_cooking_65_4_cam02_9': 2590590.0,
 'iiith_cooking_65_4_cam02_11': 1437319.0,
 'iiith_cooking_65_4_cam02_12': 663031.0,
 'iiith_cooking_65_4_cam02_13': 556351.0,
 'iiith_cooking_65_4_cam02_14': 304927.0,
 'iiith_cooking_65_4_cam02_15': 2663122.0,
 'iiith_cooking_65_4_cam03_0': 496499.0,
 'iiith_cooking_65_4_cam03_1': 1427273.0,
 'iiith_cooking_65_4_cam03_2': 93329.0,
 'iiith_cooking_65_4_cam03_3': 1528981.0,
 'iiith_cooking_65_4_cam03_4': 171641.0,
 'iiith_cooking_65_4_cam03_5': 697241.0,
 'iiith_cooking_65_4_cam03_7': 1215311.0,
 'iiith_cooking_65_4_cam03_8': 139370.0,
 'iiith_cooking_65_4_cam03_9': 1197878.0,
 'iiith_cooking_65_4_cam03_10': 1363139.0,
 'iiith_cooking_65_4_cam03_11': 382687.0,
 'iiith_cooking_65_4_cam03_12': 230526.0,
 'iiith_cooking_65_4_cam02_7': 1701849.0,
 'iiith_cooking_65_4_cam02_6': 197469.0,
 'iiith_cooking_65_4_cam02_5': 291478.0,
 'iiith_cooking_65_4_cam02_4': 893312.0

In [4]:
keystep_test.head()

,video_id,duration,narration_frame,narration,verb_class,noun_class,start_frame,end_frame,clip_id
0,georgiatech_cooking_02_01_2_cam01,643.733333,442,Read the paper with tomato and egg recipe,['read'],"['paper', 'tomato', 'egg', 'recipe']",40,845,georgiatech_cooking_02_01_2_cam01_0
1,georgiatech_cooking_02_01_2_cam01,643.733333,1203,Crack an egg into a mixing bowl,[],"['crack', 'egg', 'mixing', 'bowl']",845,1561,georgiatech_cooking_02_01_2_cam01_1
2,georgiatech_cooking_02_01_2_cam01,643.733333,1676,Heat the skillet over a medium heat,[],"['heat', 'skillet', 'medium', 'heat']",1599,1754,georgiatech_cooking_02_01_2_cam01_2
3,georgiatech_cooking_02_01_2_cam01,643.733333,1959,Add water into a mixing bowl,[],"['add', 'water', 'mixing', 'bowl']",1754,2165,georgiatech_cooking_02_01_2_cam01_3
4,georgiatech_cooking_02_01_2_cam01,643.733333,2488,Whisk the egg mixture with a spoon,[],"['egg', 'mixture', 'spoon']",2166,2809,georgiatech_cooking_02_01_2_cam01_4


In [5]:
keystep_test['SAM'] = keystep_test['clip_id'].map(gsam_map)
# Filter the DataFrame to only include rows where 'scalar' is not NaN
filtered_df = keystep_test[keystep_test['SAM'].notna()]

In [6]:
filtered_df

,video_id,duration,narration_frame,narration,verb_class,noun_class,start_frame,end_frame,clip_id,SAM
1540,iiith_cooking_120_2_cam01,209.7,122,turn on the stove with a lighter,[],"['turn', 'stove', 'lighter']",21,223,iiith_cooking_120_2_cam01_0,1383566.0
1541,iiith_cooking_120_2_cam01,209.7,281,place the skillet on the stove,[],"['place', 'skillet', 'stove']",224,337,iiith_cooking_120_2_cam01_1,5002704.0
1542,iiith_cooking_120_2_cam01,209.7,390,get a spoon from the cupboard,['get'],"['spoon', 'cupboard']",353,426,iiith_cooking_120_2_cam01_2,4689725.0
1543,iiith_cooking_120_2_cam01,209.7,478,get a bowl from the cupboard,['get'],"['bowl', 'cupboard']",426,529,iiith_cooking_120_2_cam01_3,4889472.0
1544,iiith_cooking_120_2_cam01,209.7,587,get an egg from the crate,['get'],"['egg', 'crate']",529,645,iiith_cooking_120_2_cam01_4,235845.0
...,...,...,...,...,...,...,...,...,...,...
15573,upenn_0711_Cooking_6_5_aria01_214-1,246.7,5450,Dip the tea bag in the hot water,[],"['dip', 'tea', 'bag', 'water']",5067,5832,upenn_0711_Cooking_6_5_aria01_214-1_15,96868.0
15574,upenn_0711_Cooking_6_5_aria01_214-1,246.7,5917,Get milk container from the kitchen countertop,[],"['get', 'milk', 'container', 'kitchen', 'count...",5833,6000,upenn_0711_Cooking_6_5_aria01_214-1_16,1796940.0
15575,upenn_0711_Cooking_6_5_aria01_214-1,246.7,6230,Pour milk into the tea in the cup,[],"['pour', 'milk', 'tea', 'cup']",6000,6460,upenn_0711_Cooking_6_5_aria01_214-1_17,13718.0
15576,upenn_0711_Cooking_6_5_aria01_214-1,246.7,6558,Place the milk container in the refrigerator,['place'],"['milk', 'container', 'refrigerator']",6460,6657,upenn_0711_Cooking_6_5_aria01_214-1_18,73496.0
